In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
import gc
import math

/home/ian/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Data
Remove SalesID since it is all unique numbers

In [20]:
df_train = pd.read_csv('data/Train.csv')
df_test = pd.read_csv('data/test.csv')

del df_train['SalesID']
df_train.info()

/home/ian/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (13,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401125 entries, 0 to 401124
Data columns (total 52 columns):
SalePrice                   401125 non-null int64
MachineID                   401125 non-null int64
ModelID                     401125 non-null int64
datasource                  401125 non-null int64
auctioneerID                380989 non-null float64
YearMade                    401125 non-null int64
MachineHoursCurrentMeter    142765 non-null float64
UsageBand                   69639 non-null object
saledate                    401125 non-null object
fiModelDesc                 401125 non-null object
fiBaseModel                 401125 non-null object
fiSecondaryDesc             263934 non-null object
fiModelSeries               56908 non-null object
fiModelDescriptor           71919 non-null object
ProductSize                 190350 non-null object
fiProductClassDesc          401125 non-null object
state                       401125 non-null object
ProductGroup                

SalePrice is the dependent variable

In [21]:
x = df_train

y = df_train.pop('SalePrice').values
print(x.shape, y.shape)

(401125, 51) (401125,)


In [22]:
train_columns = x.columns

for c in x.dtypes[x.dtypes == object].index.values:
    x[c] = (x[c] == True)

In [23]:
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.15, random_state=42)

# XGBoost

In [24]:
d_train = xgb.DMatrix(x_train, label=y_train)
d_valid = xgb.DMatrix(x_valid, label=y_valid)

In [25]:
from sklearn.model_selection import cross_val_score

params = {}
params['eta'] = 0.02
params['objective'] = 'reg:linear'
params['eval_metric'] = 'mae'
params['max_depth'] = 4
params['silent'] = 1

watchlist = [(d_train, 'train'), (d_valid, 'valid')]
clf_xgb = xgb.train(params, d_train, 10000, watchlist, early_stopping_rounds=100, verbose_eval=10)

[0]	train-mae:30491.6	valid-mae:30393.1
Multiple eval metrics have been passed: 'valid-mae' will be used for early stopping.

Will train until valid-mae hasn't improved in 100 rounds.
[10]	train-mae:24919.7	valid-mae:24816.7
[20]	train-mae:20597	valid-mae:20490.7
[30]	train-mae:17640.9	valid-mae:17525
[40]	train-mae:15730.1	valid-mae:15602.6
[50]	train-mae:14517.3	valid-mae:14385.4
[60]	train-mae:13724.8	valid-mae:13593.1
[70]	train-mae:13216.8	valid-mae:13085.9
[80]	train-mae:12879.9	valid-mae:12751.8
[90]	train-mae:12654.9	valid-mae:12531.6
[100]	train-mae:12507.1	valid-mae:12388.9
[110]	train-mae:12392.4	valid-mae:12280
[120]	train-mae:12307.3	valid-mae:12198.6
[130]	train-mae:12261.7	valid-mae:12152.4
[140]	train-mae:12218.9	valid-mae:12110.8
[150]	train-mae:12190	valid-mae:12085.2
[160]	train-mae:12142.4	valid-mae:12039
[170]	train-mae:12098	valid-mae:11995.2
[180]	train-mae:12070.7	valid-mae:11969.8
[190]	train-mae:12022	valid-mae:11921.6
[200]	train-mae:11971.7	valid-mae:11871.6

In [26]:
x_test = df_test[train_columns]
for c in x_test.dtypes[x_test.dtypes == object].index.values:
    x_test[c] = (x_test[c] == True)

# del df_test, sample; gc.collect()

d_test = xgb.DMatrix(x_test)

In [27]:
p_test = clf_xgb.predict(d_test)

In [28]:
sub = pd.read_csv('data/median_benchmark.csv')
for c in sub.columns[sub.columns != 'SalesId']:
    sub[c] = p_test

In [31]:
sub.to_csv('data/xgb_starter.csv', index=False)

In [32]:
def score(predictions, target):
    log_diff = np.log(predictions+1) - np.log(target+1)
    return np.sqrt(np.mean(log_diff**2))

In [49]:
benchmark = pd.read_csv('data/median_benchmark.csv')
benchmark.set_index('SalesID')
predictions = pd.read_csv('data/xgb_starter.csv')
predictions.set_index('SalesID')
test_solution = pd.read_csv('data/do_not_open/test_soln.csv')
test_solution.set_index('SalesID')
score_benchmark = score(benchmark.SalePrice, test_solution.SalePrice)
score_xgb = score(predictions.SalePrice, test_solution.SalePrice)
print('Benchmark Score: ', score_benchmark)
print('XGBoost Score  : ', score_xgb)

Benchmark Score:  0.74414067087
XGBoost Score  :  0.345689191863


/home/ian/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: RuntimeWarning: invalid value encountered in log
  from ipykernel import kernelapp as app


# lightGBM
Now we will give lightGMB a try

In [41]:
import lightgbm as lgb

d_train = lgb.Dataset(x_train, label=y_train)
d_valid = lgb.Dataset(x_valid, label=y_valid)

params = {}
params['learning_rate'] = 0.002
params['boosting_type'] = 'gbdt'
params['objective'] = 'regression'
params['metric'] = 'mae'
params['sub_feature'] = 0.5
params['num_leaves'] = 60
params['min_data'] = 500
params['min_hessian'] = 1

watchlist = [d_valid]
clf_lgb = lgb.train(params, d_train, 10000, watchlist)

[1]	valid_0's l1: 17469
[2]	valid_0's l1: 17450.7
[3]	valid_0's l1: 17433.4
[4]	valid_0's l1: 17415.2
[5]	valid_0's l1: 17399.3
[6]	valid_0's l1: 17383.5
[7]	valid_0's l1: 17382.6
[8]	valid_0's l1: 17381.6
[9]	valid_0's l1: 17363.5
[10]	valid_0's l1: 17345.5
[11]	valid_0's l1: 17329.3
[12]	valid_0's l1: 17313.2
[13]	valid_0's l1: 17295.7
[14]	valid_0's l1: 17290.4
[15]	valid_0's l1: 17286.6
[16]	valid_0's l1: 17268.8
[17]	valid_0's l1: 17267.8
[18]	valid_0's l1: 17266.9
[19]	valid_0's l1: 17261
[20]	valid_0's l1: 17260
[21]	valid_0's l1: 17256.9
[22]	valid_0's l1: 17256
[23]	valid_0's l1: 17252.2
[24]	valid_0's l1: 17247.5
[25]	valid_0's l1: 17244
[26]	valid_0's l1: 17243.1
[27]	valid_0's l1: 17225.4
[28]	valid_0's l1: 17220.2
[29]	valid_0's l1: 17216.2
[30]	valid_0's l1: 17200.1
[31]	valid_0's l1: 17196.1
[32]	valid_0's l1: 17192.1
[33]	valid_0's l1: 17191.1
[34]	valid_0's l1: 17190.2
[35]	valid_0's l1: 17175
[36]	valid_0's l1: 17157.4
[37]	valid_0's l1: 17141.5
[38]	valid_0's l1: 171

In [43]:
clf_lgb.reset_parameter({"num_threads":4})
p_test = clf_lgb.predict(x_test)

In [44]:
sub = pd.read_csv('data/median_benchmark.csv')
for c in sub.columns[sub.columns != 'SalesId']:
    sub[c] = p_test

In [45]:
sub.to_csv('data/lgb_starter.csv', index=False)

In [48]:
benchmark = pd.read_csv('data/median_benchmark.csv')
benchmark.set_index('SalesID')
predictions = pd.read_csv('data/lgb_starter.csv')
predictions.set_index('SalesID')
test_solution = pd.read_csv('data/do_not_open/test_soln.csv')
test_solution.set_index('SalesID')
score_benchmark = score(benchmark.SalePrice, test_solution.SalePrice)
score_lgb = score(predictions.SalePrice, test_solution.SalePrice)
print('Benchmark Score: ', score_benchmark)
print('lightGBM Score  : ', score_lgb)

Benchmark Score:  0.74414067087
lightGBM Score  :  0.448674247197


/home/ian/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: RuntimeWarning: invalid value encountered in log
  from ipykernel import kernelapp as app
